In [1]:
% pylab inline
import numpy as np
import pandas as pd
import os
import cv2

Populating the interactive namespace from numpy and matplotlib


In [2]:
read_data=pd.read_csv('flickr_logos_27_dataset/flickr_logos_27_dataset_query_set_annotation.txt',sep="\t",header=None)

In [3]:
read_data.columns=["ID","labels"]

In [4]:
read_data

,ID,labels
0,2403695909.jpg,Adidas
1,2912587920.jpg,Adidas
2,3441398196.jpg,Adidas
3,4605630935.jpg,Adidas
4,4606245138.jpg,Adidas
5,161270014.jpg,Apple
6,3276332721.jpg,Apple
7,51409302.jpg,Apple
8,3842830049.jpg,Apple
9,4748791597.jpg,Apple


In [5]:
from scipy.misc import *

temp=[]
for img_name in read_data.ID:
    img_path=os.path.join('flickr_logos_27_dataset/flickr_logos_27_dataset_images',img_name)
    img=imread(img_path)
    img=imresize(img,(28,28))
    img=img.astype('float32')
    temp.append(img)

import numpy as np

data=np.stack(temp)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [6]:
labels=read_data["labels"]

normalized_data=data/255

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# Loop through the splits (only one)
for train_indices, test_indices in splitter.split(normalized_data, labels):
    # Select the train and test data
    x_train, y_train = normalized_data[train_indices], labels[train_indices]
    x_test, y_test = normalized_data[test_indices], labels[test_indices]


In [8]:
from sklearn.preprocessing import LabelEncoder
import keras
lb = LabelEncoder()
train_labels = lb.fit_transform(y_train)
y_train = keras.utils.np_utils.to_categorical(train_labels)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(216, 28, 28, 3)
(216, 28)
(54, 28, 28, 3)
(54,)


In [10]:
from keras.models import Sequential
from keras.layers import *

model=Sequential()
model.add(Conv2D(28,(3,3),padding='same',activation='relu',input_shape=(28,28,3)))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(28,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
from keras.preprocessing.image import ImageDataGenerator 
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
vertical_flip=False) # randomly flip images

In [12]:
gen_flow=datagen.flow(x_train,y_train,batch_size=32,seed=0)

In [13]:
model.fit_generator(gen_flow,steps_per_epoch=len(x_train) / 128, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s - loss: 3.3432 - acc: 0.0312
Epoch 2/200
1/1 [==============================] - 0s - loss: 3.8729 - acc: 0.6250
Epoch 3/200
1/1 [==============================] - 0s - loss: 3.9727 - acc: 0.5000
Epoch 4/200
1/1 [==============================] - 0s - loss: 2.9871 - acc: 0.4375
Epoch 5/200
1/1 [==============================] - 0s - loss: 3.2206 - acc: 0.4062
Epoch 6/200
1/1 [==============================] - 0s - loss: 3.2101 - acc: 0.4688
Epoch 7/200
1/1 [==============================] - 0s - loss: 3.2708 - acc: 0.4583
Epoch 8/200
1/1 [==============================] - 0s - loss: 3.2217 - acc: 0.4375
Epoch 9/200
1/1 [==============================] - 0s - loss: 3.1070 - acc: 0.5625
Epoch 10/200
1/1 [==============================] - 0s - loss: 3.0496 - acc: 0.5000
Epoch 11/200
1/1 [==============================] - 0s - loss: 2.9560 - acc: 0.5625
Epoch 12/200
1/1 [==============================] - 0s - loss: 2.9624 - acc: 0.4688
E

1/1 [==============================] - 0s - loss: 1.5970 - acc: 0.5833
Epoch 99/200
1/1 [==============================] - 0s - loss: 1.5709 - acc: 0.5938
Epoch 100/200
1/1 [==============================] - 0s - loss: 1.4628 - acc: 0.6250
Epoch 101/200
1/1 [==============================] - 0s - loss: 1.4081 - acc: 0.5625
Epoch 102/200
1/1 [==============================] - 0s - loss: 2.1030 - acc: 0.3750
Epoch 103/200
1/1 [==============================] - 0s - loss: 1.5687 - acc: 0.6250
Epoch 104/200
1/1 [==============================] - 0s - loss: 2.0905 - acc: 0.3438
Epoch 105/200
1/1 [==============================] - 0s - loss: 1.6586 - acc: 0.5417
Epoch 106/200
1/1 [==============================] - 0s - loss: 1.7472 - acc: 0.5312
Epoch 107/200
1/1 [==============================] - 0s - loss: 1.7033 - acc: 0.5000
Epoch 108/200
1/1 [==============================] - 0s - loss: 1.3201 - acc: 0.5938
Epoch 109/200
1/1 [==============================] - 0s - loss: 1.7768 - acc: 0.

1/1 [==============================] - 0s - loss: 1.0288 - acc: 0.6250
Epoch 195/200
1/1 [==============================] - 0s - loss: 0.5721 - acc: 0.8438
Epoch 196/200
1/1 [==============================] - 0s - loss: 0.8809 - acc: 0.7083
Epoch 197/200
1/1 [==============================] - 0s - loss: 0.6712 - acc: 0.8125
Epoch 198/200
1/1 [==============================] - 0s - loss: 0.9776 - acc: 0.6875
Epoch 199/200
1/1 [==============================] - 0s - loss: 0.6837 - acc: 0.7812
Epoch 200/200
1/1 [==============================] - 0s - loss: 0.6241 - acc: 0.8125


In [14]:
pred = model.predict_classes(x_test)
print np.unique(pred)
pred = lb.inverse_transform(pred)

54/54 [==============================] - 0s     
[ 5  7  9 13 26 27]


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
from sklearn.metrics import accuracy_score

accuracy_score(pred,y_test)

0.5185185185185185